<a href="https://colab.research.google.com/github/Unalunapark/practice/blob/main/%EC%84%B9%EC%85%9812_2%ED%9A%8C_%EA%B8%B0%EC%B6%9C%EC%9C%A0%ED%98%95(%EC%9E%91%EC%97%85%ED%98%952).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형2
- 빅데이터 분석기사 2회 실기
- 데이터 다운로드 : https://www.kaggle.com/prachi13/customer-analytics
- 데이터 파일 명 : Train.csv (파일명이 다를 경우 파일명을 수정해주세요)
- (데이터가 삭제될 시 수업노트에 유사 데이터 셋을 올려두겠습니다)


## 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
- 제공된 데이터
    - X_train.csv
    - y_train.csv
    - X_test.csv

- 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
- 0 정시 도착, 1 정시 도착하지 않음

```
ID,Reached.on.Time_Y.N
3500,0.267
3501,0.578
3502,0.123

```

- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

- 데이터 파일 읽기 예제

```
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")
```

- csv파일 생성 예시

```
pd.DataFrame({'ID': X_test['cust_id'], 'Reached.on.Time_Y.N': pred}).to_csv('003000000.csv', index=False)
```


In [42]:
# 시험환경 세팅 3개 파일 생성 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("/content/drive/MyDrive/데이터모음/★데이터모음/☆인프런 데이터/archive/Train.csv") # 파일명이 다를 경우 파일명을 수정해주세요
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

X_train.shape, y_train.shape, X_test.shape

((8799, 11), (8799, 2), (2200, 11))

## EDA

In [45]:
# 데이터 확인(X_train)
X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,B,Ship,4,1,284,4,low,F,4,4795
1,10849,D,Ship,3,2,201,4,medium,M,10,5565
2,9478,B,Ship,4,4,181,3,medium,F,9,4425
3,2968,B,Ship,6,2,236,4,low,F,61,3140
4,6964,B,Road,5,1,282,4,low,F,9,4766


In [46]:
# 데이터 확인(y_train)
y_train.head()
#0: 정시도착, 1: 정시 미도착

,ID,Reached.on.Time_Y.N
0,8440,1
1,10849,0
2,9478,1
3,2968,1
4,6964,0


In [47]:
# 결측치 확인 train 없음

In [48]:
# 결측치 확인 test 없음


In [49]:
# 타겟(레이블) 확인 0 정시 도착, 1 정시 도착하지 않음
y_train['Reached.on.Time_Y.N'].value_counts()
#0: 정시도착 3563 < 1: 정시 미도착 5236

Reached.on.Time_Y.N
1    5236
0    3563
Name: count, dtype: int64

In [50]:
# 데이터 타입 확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   8799 non-null   int64 
 1   Warehouse_block      8799 non-null   object
 2   Mode_of_Shipment     8799 non-null   object
 3   Customer_care_calls  8799 non-null   int64 
 4   Customer_rating      8799 non-null   int64 
 5   Cost_of_the_Product  8799 non-null   int64 
 6   Prior_purchases      8799 non-null   int64 
 7   Product_importance   8799 non-null   object
 8   Gender               8799 non-null   object
 9   Discount_offered     8799 non-null   int64 
 10  Weight_in_gms        8799 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 756.3+ KB


In [51]:
# 기초 통계 획인
X_train.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000,8799.000000
mean,5514.696556,4.051369,2.991817,209.800318,3.561882,13.293215,3633.145130
std,3175.107399,1.141739,1.411938,48.186779,1.518472,16.112977,1637.717073
min,1.000000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000
25%,2749.500000,3.000000,2.000000,169.000000,3.000000,4.000000,1837.000000
50%,5519.000000,4.000000,3.000000,213.000000,3.000000,7.000000,4152.000000
75%,8253.000000,5.000000,4.000000,250.000000,4.000000,10.000000,5055.000000
max,10999.000000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000


In [52]:
# 기초 통계 획인 object
X_train.describe(include='O')
#unique값이 트레인과 테스트가 같지 않으면 붙였다가 떼어야 함

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,8799,8799,8799,8799
unique,5,3,3,2
top,F,Ship,low,F
freq,2929,5972,4225,4450


In [53]:
# 기초 통계 획인 object
X_test.describe(include='O')

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,2200,2200,2200,2200
unique,5,3,3,2
top,F,Ship,low,M
freq,737,1490,1072,1105


In [54]:
# [Tip] object 타입 고유값 개수 확인
X_train[['Warehouse_block',	'Mode_of_Shipment','Product_importance',	'Gender']]

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
0,B,Ship,low,F
1,D,Ship,medium,M
2,B,Ship,medium,F
3,B,Ship,low,F
4,B,Road,low,F
...,...,...,...,...
8794,D,Ship,high,F
8795,C,Ship,low,M
8796,F,Ship,low,F
8797,A,Flight,medium,F


In [55]:
X_train[['Warehouse_block',	'Mode_of_Shipment','Product_importance',	'Gender']].nunique()

Warehouse_block       5
Mode_of_Shipment      3
Product_importance    3
Gender                2
dtype: int64

In [56]:
X_train[['Warehouse_block',	'Mode_of_Shipment','Product_importance',	'Gender']].value_counts()

Warehouse_block  Mode_of_Shipment  Product_importance  Gender
F                Ship              low                 M         504
                                                       F         466
                                   medium              F         459
                                                       M         398
A                Ship              low                 F         258
                                                                ... 
D                Flight            high                F           9
C                Road              high                F           9
B                Flight            high                F           8
                                                       M           8
D                Road              high                F           5
Name: count, Length: 90, dtype: int64

In [57]:
X_train['Warehouse_block'].value_counts()

Warehouse_block
F    2929
A    1488
C    1463
D    1460
B    1459
Name: count, dtype: int64

In [ ]:
# X데이터와 y데이터 합치기
print(help(pd.concat))

In [59]:
df=pd.concat([X_train, y_train['Reached.on.Time_Y.N']], axis=1)
print(df.head())
print(df.info())

      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0   8440               B             Ship                    4   
1  10849               D             Ship                    3   
2   9478               B             Ship                    4   
3   2968               B             Ship                    6   
4   6964               B             Road                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                1                  284                4                low   
1                2                  201                4             medium   
2                4                  181                3             medium   
3                2                  236                4                low   
4                1                  282                4                low   

  Gender  Discount_offered  Weight_in_gms  Reached.on.Time_Y.N  
0      F                 4           4795                    1 

In [60]:
# idea? 할인율이 도착에 영향을 미칠까?
#9   Discount_offered     8799 non-null   int64  -> 11  Reached.on.Time_Y.N  8799 non-null   int64
df[['Discount_offered','Reached.on.Time_Y.N']]
#0: 정시도착 3563 < 1: 정시 미도착 5236

,Discount_offered,Reached.on.Time_Y.N
0,4,1
1,10,0
2,9,1
3,61,1
4,9,0
...,...,...
8794,6,1
8795,7,0
8796,63,1
8797,17,1


In [61]:
df['Discount_offered'] > 5

0       False
1        True
2        True
3        True
4        True
        ...  
8794     True
8795     True
8796     True
8797     True
8798    False
Name: Discount_offered, Length: 8799, dtype: bool

In [62]:
con=df['Discount_offered'] > 30
sum(con)
# 5이상: 5480
# 30이상: 1357

1357

In [63]:
df[con]

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
3,2968,B,Ship,6,2,236,4,low,F,61,3140,1
10,3058,B,Ship,5,5,257,4,medium,M,31,2040,1
14,824,F,Flight,5,3,178,2,medium,F,60,1830,1
18,2570,F,Ship,5,2,263,2,medium,F,63,1254,1
32,1544,F,Ship,3,4,212,2,medium,F,56,3976,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8750,2095,D,Ship,3,5,272,10,medium,M,59,2182,1
8753,534,F,Road,3,1,141,3,medium,F,39,1838,1
8756,1600,B,Ship,3,3,198,3,low,M,49,2446,1
8777,2146,B,Ship,4,4,181,3,low,M,44,2070,1


In [64]:
df[con]['Reached.on.Time_Y.N']

3       1
10      1
14      1
18      1
32      1
       ..
8750    1
8753    1
8756    1
8777    1
8796    1
Name: Reached.on.Time_Y.N, Length: 1357, dtype: int64

In [65]:
sum(df[con]['Reached.on.Time_Y.N']) #1 지각 을 합친값=3673 5이상, 30이상 1357

1357

## 데이터 전처리 및 피처엔지니어링

In [66]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
cols=X_train.select_dtypes(include='O').columns
print(cols)

Index(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], dtype='object')


In [ ]:
print(help(LabelEncoder))

In [68]:
for col in cols:
    le=LabelEncoder()
    X_train[col]=le.fit_transform(X_train[col])
    X_test[col]=le.transform(X_test[col])

X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,1,2,4,1,284,4,1,0,4,4795
1,10849,3,2,3,2,201,4,2,1,10,5565
2,9478,1,2,4,4,181,3,2,0,9,4425
3,2968,1,2,6,2,236,4,1,0,61,3140
4,6964,1,1,5,1,282,4,1,0,9,4766


#네번 작성해도 됨

col='Warehouse_block'
le=LabelEncoder()
X_train[col]=le.fit_transform(X_train[col])
X_test[col]=le.transform(X_test[col])


In [ ]:
# trainID 삭제, testID 값만 옮겨둠
help(X_train.drop)

In [70]:
X_train=X_train.drop('ID', axis=1)
X_test_id=X_test.pop('ID') #ID값을 넘겨주고나서, 삭제하기

## 모델 및 평가

In [30]:
# 라이브러리 불러오기
# 0 정시 도착, 1 정시 도착하지 않음
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
# rf=RandomForestClassifier()
# rf.fit()

In [71]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Warehouse_block      8799 non-null   int64
 1   Mode_of_Shipment     8799 non-null   int64
 2   Customer_care_calls  8799 non-null   int64
 3   Customer_rating      8799 non-null   int64
 4   Cost_of_the_Product  8799 non-null   int64
 5   Prior_purchases      8799 non-null   int64
 6   Product_importance   8799 non-null   int64
 7   Gender               8799 non-null   int64
 8   Discount_offered     8799 non-null   int64
 9   Weight_in_gms        8799 non-null   int64
dtypes: int64(10)
memory usage: 687.5 KB


In [72]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Warehouse_block      2200 non-null   int64
 1   Mode_of_Shipment     2200 non-null   int64
 2   Customer_care_calls  2200 non-null   int64
 3   Customer_rating      2200 non-null   int64
 4   Cost_of_the_Product  2200 non-null   int64
 5   Prior_purchases      2200 non-null   int64
 6   Product_importance   2200 non-null   int64
 7   Gender               2200 non-null   int64
 8   Discount_offered     2200 non-null   int64
 9   Weight_in_gms        2200 non-null   int64
dtypes: int64(10)
memory usage: 172.0 KB


In [74]:
y_train.head()

,ID,Reached.on.Time_Y.N
0,8440,1
1,10849,0
2,9478,1
3,2968,1
4,6964,0


In [ ]:
import sklearn
dir(sklearn)
#'metrics',  'model_selection',

In [ ]:
help(sklearn.model_selection)
#  train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

In [82]:
# 검증 데이터 분리
from sklearn.model_selection import train_test_split
xtr, xval, ytr, yval=train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size=0.2, random_state=2021)
xtr.shape, xval.shape, ytr.shape, yval.shape
#((7039, 10), (1760, 10), (7039,), (1760,))

((7039, 10), (1760, 10), (7039,), (1760,))

In [32]:
# 로지스틱 회귀


In [33]:
# 의사결정나무


## 문제확인
학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

In [83]:
# 랜덤포레스트
# 0 정시 도착, 1 정시 도착하지 않음
#1일 확률 [1]
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
rf=RandomForestClassifier()
rf.fit(xtr, ytr)
pred=rf.predict_proba(xval)
print(pred)

[[0.54 0.46]
 [0.55 0.45]
 [0.59 0.41]
 ...
 [0.19 0.81]
 [0.   1.  ]
 [0.14 0.86]]


In [35]:
# XGBOOST


#평가 ROC_AUC

In [87]:
from sklearn.metrics import roc_auc_score
roc_auc_score(yval, pred[:,1])

0.7243024190719787

## 예측 및 제출

In [98]:
# 예측
pred=rf.predict_proba(X_test)
print(pred)

[[0.63 0.37]
 [0.51 0.49]
 [0.58 0.42]
 ...
 [0.58 0.42]
 [0.65 0.35]
 [0.52 0.48]]


In [99]:
print(pred[:,1])

[0.37 0.49 0.42 ... 0.42 0.35 0.48]


## 제출형식


```
ID,Reached.on.Time_Y.N
3500,0.267
3501,0.578
3502,0.123
```



In [91]:
# 데이터프레임 만들기
submit=pd.DataFrame(
    {
        'ID':X_test_id,
        'Reached.on.Time_Y.N':pred[:,1]
    }
)

In [92]:
# 데이터 확인
print(submit)

         ID  Reached.on.Time_Y.N
0      8285                 0.37
1     10192                 0.49
2      8675                 0.42
3      5753                 0.44
4       448                 0.96
...     ...                  ...
2195   7193                 0.33
2196   4445                 0.35
2197   7390                 0.42
2198   9977                 0.35
2199   5696                 0.48

[2200 rows x 2 columns]


In [100]:
help(submit.to_csv)

Help on method to_csv in module pandas.core.generic:

to_csv(path_or_buf: 'FilePath | WriteBuffer[bytes] | WriteBuffer[str] | None' = None, sep: 'str' = ',', na_rep: 'str' = '', float_format: 'str | Callable | None' = None, columns: 'Sequence[Hashable] | None' = None, header: 'bool_t | list[str]' = True, index: 'bool_t' = True, index_label: 'IndexLabel | None' = None, mode: 'str' = 'w', encoding: 'str | None' = None, compression: 'CompressionOptions' = 'infer', quoting: 'int | None' = None, quotechar: 'str' = '"', lineterminator: 'str | None' = None, chunksize: 'int | None' = None, date_format: 'str | None' = None, doublequote: 'bool_t' = True, escapechar: 'str | None' = None, decimal: 'str' = '.', errors: 'str' = 'strict', storage_options: 'StorageOptions' = None) -> 'str | None' method of pandas.core.frame.DataFrame instance
    Write object to a comma-separated values (csv) file.
    
    Parameters
    ----------
    path_or_buf : str, path object, file-like object, or None, defaul


##submit.to_csv

```
 >>> df.to_csv(index=False)
    'name,mask,weapon\nRaphael,red,sai\nDonatello,purple,bo staff\n'
    
    Create 'out.zip' containing 'out.csv'
    
    >>> compression_opts = dict(method='zip',
    ...                         archive_name='out.csv')  # doctest: +SKIP
    >>> df.to_csv('out.zip', index=False,
    ...           compression=compression_opts)  # doctest: +SKIP
```   



In [101]:
# CSV파일 만들기
submit.to_csv('00000.csv', index=False) # '수험번호.csv'

##  제대로 제출 되었는지 확인

In [102]:
pd.read_csv('/content/00000.csv')

,ID,Reached.on.Time_Y.N
0,8285,0.37
1,10192,0.49
2,8675,0.42
3,5753,0.44
4,448,0.96
...,...,...
2195,7193,0.33
2196,4445,0.35
2197,7390,0.42
2198,9977,0.35


## 체점
- 수험자는 알 수 없는 부분임

In [104]:
you = pd.read_csv("00000.csv")
print("당신의 score는?:",roc_auc_score(y_test['Reached.on.Time_Y.N'], you['Reached.on.Time_Y.N']))
#0.732290234282947

당신의 score는?: 0.732290234282947
